# Gerador de horóscopo

Inputs do modelo:
Utiliza como referência para as previsões os seguintes sites:
- João Bidu
- Terra
- UOL
- Personare

Outputs do modelo:
Texto contendo entre 100 e 120 palavras no banco de dados

In [19]:
import openai
from dotenv import load_dotenv
import os
import requests
from datetime import datetime, time, timezone, timedelta
import random
from bs4 import BeautifulSoup
import json

In [20]:
load_dotenv()
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [21]:
def get_horoscope_prediction_joao_bidu(sign):
    try:
        url =  f'https://joaobidu.com.br/horoscopo/signos/previsao-{sign}/'
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        elemento = soup.find('div', class_='theiaPostSlider_preloadedSlide')
        previsao = elemento.find('div', class_='zoxrel left').text.split('\n')[0][7:-1]
    except:
        previsao = ''

    return previsao

In [22]:
def get_horoscope_prediction_terra(sign):
    try:
        url =  f'https://www.terra.com.br/vida-e-estilo/horoscopo/signos/{sign}/'
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        elemento = soup.find('div', class_='horoscope--content__embed')
        previsao = elemento.find('p').text
    except:
        previsao = ''

    return previsao

In [23]:
def get_horoscope_prediction_uol(sign):
    try:
        url =  f'https://www.uol.com.br/universa/horoscopo/{sign}/horoscopo-do-dia/'
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        elemento = soup.find('div', class_='horoscope-open-content')
        previsao = elemento.find('p').text#.split('\n')[0][7:-1]
    except:
        previsao = ''

    return previsao

In [24]:
def get_horoscope_prediction_personare(sign):
    try:
        url =  f'https://www.personare.com.br/horoscopo-do-dia/{sign}'
        response = requests.get(url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        elemento = soup.find('div', class_='styles__Text-sc-1ryixz1-3')
        previsao = elemento.find('p').text#.split('\n')[0][7:-1]
    except:
        previsao = ''
    
    return previsao

In [25]:
def get_horoscope_prediction(sign):

    previsao_joaobidu = get_horoscope_prediction_joao_bidu(sign)
    previsao_terra = get_horoscope_prediction_terra(sign)
    previsao_uol = get_horoscope_prediction_uol(sign)
    previsao_personare = get_horoscope_prediction_personare(sign)
    
    predictions = f'Previsão 1: {previsao_joaobidu}\n\nPrevisão 2: {previsao_terra}\n\nPrevisão 3: {previsao_uol}\n\nPrevisão 4: {previsao_personare}' 

    return predictions

In [26]:
def save_horoscope(sign, prediction):
    url = 'http://localhost:3000/api/horoscope'
    
    now = datetime.now(timezone.utc)
    tomorrow = now + timedelta(days=1)

    payload = {
        "uid": "abcdefghijklmnopq-cartomante",
        "supportUid": "johannesadmin",
        "h1": sign, 
        "text": prediction, 
    }
    response = requests.post(url, json=payload)

    if response.status_code != 200:
        print(f'Erro: {response.status_code}')

In [27]:
def create_daily_horoscope():
    signos = [
        {"name": "Áries", "fromTo": "21-mar a 20-abr", "reference": "aries"},
        {"name": "Touro", "fromTo": "21-abr a 20-mai", "reference": "touro"},
        {"name": "Gêmeos", "fromTo": "21-mai a 20-jun", "reference": "gemeos"},
        {"name": "Câncer", "fromTo": "21-jun a 21-jul", "reference": "cancer"},
        {"name": "Leão", "fromTo": "22-jul a 22-ago", "reference": "leao"},
        {"name": "Virgem", "fromTo": "23-ago a 22-set", "reference": "virgem"},
        {"name": "Libra", "fromTo": "23-set a 22-out", "reference": "libra"},
        {"name": "Escorpião", "fromTo": "23-out a 21-nov", "reference": "escorpiao"},
        {"name": "Sagitário", "fromTo": "22-nov a 21-dez", "reference": "sagitario"},
        {"name": "Capricórnio", "fromTo": "22-dez a 20-jan", "reference": "capricornio"},
        {"name": "Aquário", "fromTo": "21-jan a 19-fev", "reference": "aquario"},
        {"name": "Peixes", "fromTo": "20-fev a 20-mar", "reference": "peixes"}
    ]
    
    startWith = [
        "hoje",
        "aproveite o momento para",
        "concentre-se em",
        "é o momento perfeito para",
        "preste atenção",
        "busque o equilíbrio em suas relações e tome decisões ponderadas.",
        "um período de",
        "novas aventuras e oportunidades estão prestes a surgir.",
        "trabalhe com",
        "sua",
        "permita-se",
    ]
    
    for s in signos:
        reference = get_horoscope_prediction(s["reference"])
        content = f'Qual o horóscopo de {s["name"].lower()} para hoje? Escreva um texto entre 100 e 120 palavras que será publicado em um jornal. Se necessário, considere a referêcia existente em alguns sites de prestígio \n\n{reference}\n\nComece com: "{s["name"]}, {random.choice(startWith)}"'
        message = [{
            'role': 'user',
            'content': content
        }]

        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=1000, 
            temperature=0, 
            )
        prediction = response.choices[0].message.content
        save_horoscope(s["name"], prediction)
        

In [29]:
create_daily_horoscope()